In [1]:
import pandas as pd  #データの整理
import numpy as np
df_gaia = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e9_gaia.csv")
df_ucac = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e9_ucac.csv")
df_gaia = df_gaia.rename(columns={"star_number": "star_number_gaia"})
df_gaia = df_gaia.rename(columns={"zero_mag": "zero_mag_gaia"})
df_ucac = df_ucac.rename(columns={"star_number": "star_number_ucac"})
df_ucac = df_ucac.rename(columns={"zero_mag": "zero_mag_ucac"})

columns = df_gaia.columns
df_gaia = df_gaia.values
df_gaia = np.delete(df_gaia,np.arange(141),1)
columns = np.delete(columns,np.arange(141))
df_gaia = pd.DataFrame(data=df_gaia,columns=columns)
df_concat = pd.concat([df_ucac, df_gaia], axis=1)
columns_concat = df_concat.columns
array_concat = df_concat.values

for i in  range(len(columns)):  # IDの前にある不要カラムを削除
    if columns_concat[i] == "ID":
        delete_column = i
array_concat = np.delete(array_concat,np.arange(delete_column),1)
columns_concat = np.delete(columns_concat,np.arange(delete_column))

### 14等級以上のものは一回リムーブ、IDをリチェックリストにして出す
recheck_index = np.where(array_concat[:,-6]>14.)
recheck_ID = array_concat[recheck_index,0]
recheck_ID = recheck_ID.transpose()
recheck_ID = recheck_ID.reshape(len(recheck_ID))
array_concat = np.delete(array_concat,recheck_index,0)
df_main = pd.DataFrame(data=array_concat,columns=columns_concat)
with open("recheck_list.txt", mode='w') as f:
    f.write("G_Magnitudeが14以上のID\n")
    for i in recheck_ID:
        state = str(i) + "\n"
        f.write(state)

df_main.to_csv("simultaneous_event_candidates_table_MU+Tomo-e10.csv")
print "Complete!"

Complete!
